In [1]:
import numpy as np
from skimage.transform import resize
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, ConvLSTM2D, TimeDistributed
from sklearn.externals import joblib
import pylidc as pl

Using TensorFlow backend.


In [2]:
image_size = (256, 256)
num_slices = 50

In [3]:
model = Sequential()
model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3), input_shape=(num_slices, *image_size, 1),
                     padding='same', return_sequences=True))
model.add(TimeDistributed(Conv2D(1, 1, activation='sigmoid')))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
unet = load_model('unet.hdf5')

In [5]:
scans = pl.query(pl.Scan)
n = scans.count()

In [ ]:
test = joblib.load('test_set.p')
train = list(set(range(n)) - set(test))

In [6]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    y = np.expand_dims(y, axis=-1)
    start = np.random.randint(n - num_slices)
    X = X[start:start + num_slices]
    y = y[start:start + num_slices]
    return unet.predict(X, batch_size=1)[None], y[None]

In [ ]:
num_epochs = 2
num_samples = 200

In [ ]:
for _ in range(num_samples):
    while True:
        i = np.random.choice(train)
        print(i)
        try:
            X, y = get_data(i)
            model.fit(X, y, batch_size=1, epochs=num_epochs)
            break
        except OSError:
            print('Problem getting data')

In [ ]:
model.save('unet_lstm.hdf5')